<a href="https://colab.research.google.com/github/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes/blob/main/AML_AnomalySegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real-time Anomaly Segmentation for Road Scenes**

In [1]:
!rm -r sample_data/

Existing deep neural networks, when deployed in open-world settings, perform poorly on unknown, anomaly, out-of-distribution (OoD) objects that were not present during the training. The goal of this project is to build tiny anomaly segmentation models to segment anomaly patterns. Models must be able to fit in small devices, which represents a realistic memory constraint for an edge application.

## Preparation

In [2]:
# download required packages and import useful modules
!pip3 install --quiet cityscapesscripts
!pip3 install --quiet gdown
!pip3 install --quiet numpy
!pip3 install --quiet matplotlib
!pip3 install --quiet Pillow
!pip3 install --quiet torchvision
!pip3 install --quiet visdom
!pip3 install --quiet ood_metrics

import os, sys, subprocess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.6/473.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


The following function is implemented to download the *Cityscapes* dataset in two different ways: via Google Drive (using `gdown`) or directly from the Cityscapes official website (using `csDownload`). Although the first option is preferable as it is definitely faster, direct download from the website is provided as an alternative. `gdown` may in fact raise the error *Failed to retrieve the file url* if the file we are attempting to download is exceptionally large (*e.g.* 11G), there are numerous users simultaneously trying to download it programmatically or we download it many times in a limited time. Regardless of the method used, use the conversor (available [here](https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/preparation/createTrainIdLabelImgs.py)) to generate labelTrainIds from labelIds.

In [3]:
def download_cityscapes():

    if not os.path.isdir('/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes'):
        print("Attempting to download cityscapes dataset using gdown...")

        try:
            # If check is true, and the process exits with a non-zero exit code, a CalledProcessError exception will be raised.
            subprocess.run(["gdown", "https://drive.google.com/uc?id=11gSQ9UcLCnIqmY7srG2S6EVwV3paOMEq"], check=True)
            print("Dataset downloaded successfully using gdown. Unzipping...")
            subprocess.run(["unzip", "-q", "cityscapes.zip"], check=True)
            # Use the conversor to generate labelTrainIds from labelIds
            print("Generating trainIds from labelIds...")
            !CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

        except subprocess.CalledProcessError as e:
            print("gdown failed. Attempting to download cityscapes dataset from the official website...")
            try:
              # Cityscapes credentials: (agnesere, FCSBwcVMi-u9-Zn)
              !csDownload leftImg8bit_trainvaltest.zip
              !csDownload gtFine_trainvaltest.zip

              print("Dataset downloaded successfully from the official website. Unzipping...")
              !unzip -q 'leftImg8bit_trainvaltest.zip' -d 'cityscapes'
              !unzip -o -q 'gtFine_trainvaltest.zip' -d 'cityscapes'

              print("Generating trainIds from labelIds...")
              !CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

              print("Cityscapes dataset ready")

            except Exception as e2:
                print("Failed to download the dataset using both methods.")

Download and unzip the validation dataset (*FS_LostFound_full*, *RoadAnomaly*, *RoadAnomaly21*, *RoadObsticle21*, *fs_static*), clone or update the GitHub repository (*Real-Time-Anomaly-Segmentation-for-Road-Scenes*) and download the *Cityscapes* dataset.

In [4]:
# download and unzip validation dataset
if not os.path.isdir('/content/validation_dataset'):
  !gdown 'https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta'
  !unzip -q 'Validation_Dataset.zip'
  !mkdir validation_dataset && cp -pR Validation_Dataset/* validation_dataset/ && rm -R Validation_Dataset/
  !rm 'Validation_Dataset.zip'

# clone the github repo and pull command
if not os.path.isdir('content/Real-Time-Anomaly-Segmentation-for-Road-Scenes'):
  !git clone https://github.com/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes.git
else: # if folder already present
  !git pull

%cd Real-Time-Anomaly-Segmentation-for-Road-Scenes

Downloading...
From (original): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
From (redirected): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta&confirm=t&uuid=9ea071cf-431c-4a8d-a2b6-4d372f1b02a7
To: /content/Validation_Dataset.zip
100% 329M/329M [00:07<00:00, 43.4MB/s]
Cloning into 'Real-Time-Anomaly-Segmentation-for-Road-Scenes'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 383 (delta 55), reused 50 (delta 23), pack-reused 288 (from 1)
Receiving objects: 100% (383/383), 25.49 MiB | 16.10 MiB/s, done.
Resolving deltas: 100% (214/214), done.
/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes


In [5]:
# download cityscapes dataset
download_cityscapes()

Attempting to download cityscapes dataset using gdown...
gdown failed. Attempting to download cityscapes dataset from the official website...
Cityscapes username or email address: agnesere
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: N
Download progress:  98% 10.8G/11.0G [09:41<00:10, 19.9MB/s]
Cityscapes username or email address: agnesere
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: N
Download progress: 100% 241M/241M [00:38<00:00, 6.61MB/s]
Dataset downloaded successfully from the official website. Unzipping...
Generating trainIds from labelIds...
Processing 5000 annotation files
Progress: 100.0 % Cityscapes dataset ready


## Evaluation

### Step 2A - Compute AuPRC & FPR95TPR

In [6]:
%cd eval

/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval


Perform various anomaly inferences using the pre-trained **ErfNet** model and anomaly segmentation test dataset provided. Different techniques are used (MSP, MaxLogit and MaxEntropy).

In [ ]:
methods_list = ["msp", "maxlogit", "maxentropy"]
datasets_list = os.listdir("../../validation_dataset")

for dataset in datasets_list:
  print(f"{dataset} dataset")
  for method in methods_list:
    print(f"\t- {method}")
    input_path = f"../../validation_dataset/{dataset}/images/*.*"
    !python evalAnomaly.py --input={input_path} --method={method}
  print("---------------------------------")

RoadObsticle21 dataset
	- msp
		AUPRC score: 2.712
		FPR@TPR95: 64.974
	- maxlogit
		AUPRC score: 4.627
		FPR@TPR95: 48.443
	- maxentropy
		AUPRC score: 3.052
		FPR@TPR95: 65.600
----------------------
RoadAnomaly21 dataset
	- msp
		AUPRC score: 29.100
		FPR@TPR95: 62.511
	- maxlogit
		AUPRC score: 38.320
		FPR@TPR95: 59.337
	- maxentropy
		AUPRC score: 31.005
		FPR@TPR95: 62.593
----------------------
RoadAnomaly dataset
	- msp
		AUPRC score: 12.426
		FPR@TPR95: 82.492
	- maxlogit
		AUPRC score: 15.582
		FPR@TPR95: 73.248
	- maxentropy
		AUPRC score: 12.678
		FPR@TPR95: 82.632
----------------------
fs_static dataset
	- msp
		AUPRC score: 7.470
		FPR@TPR95: 41.823
	- maxlogit
		AUPRC score: 9.499
		FPR@TPR95: 40.300
	- maxentropy
		AUPRC score: 8.826
		FPR@TPR95: 41.523
----------------------
FS_LostFound_full dataset
	- msp
		AUPRC score: 1.748
		FPR@TPR95: 50.763
	- maxlogit
		AUPRC score: 3.301
		FPR@TPR95: 45.495
	- maxentropy
		AUPRC score: 2.582
		FPR@TPR95: 50.368
-------------

### Step 2B - Compute AuPRC & FPR95TPR with temperature scaling

In [ ]:
datasets_list = os.listdir("../../validation_dataset")


In [ ]:
temperature_list = [0.5, 0.75, 1.0, 1.1, 1.2, 1.5, 2.0, 3.0, 5.0, 10.0, 50.0]

for dataset in datasets_list:
  print(f"{dataset} dataset")
  for t in temperature_list:
    print(f"\t- {t}")
    input_path = f"../../validation_dataset/{dataset}/images/*.*"
    !python evalAnomaly.py --input={input_path} --method="msp" --temperature={t}
  print("---------------------------------")

RoadAnomaly21
	- 0.5
		AUPRC score: 27.061
		FPR@TPR95: 62.731
	- 0.75
		AUPRC score: 28.156
		FPR@TPR95: 62.479
	- 1.0
		AUPRC score: 29.100
		FPR@TPR95: 62.511
	- 1.1
		AUPRC score: 29.410
		FPR@TPR95: 62.590
	- 1.2
		AUPRC score: 29.678
		FPR@TPR95: 62.724
	- 1.5
		AUPRC score: 30.258
		FPR@TPR95: 63.318
	- 2.0
		AUPRC score: 30.679
		FPR@TPR95: 64.721
	- 3.0
		AUPRC score: 30.674
		FPR@TPR95: 67.682
	- 5.0
		AUPRC score: 30.196
		FPR@TPR95: 71.594
	- 10.0
		AUPRC score: 29.526
		FPR@TPR95: 75.757
	- 50.0
		AUPRC score: 28.804
		FPR@TPR95: 80.014
FS_LostFound_full
	- 0.5
		AUPRC score: 1.280
		FPR@TPR95: 66.737
	- 0.75
		AUPRC score: 1.493
		FPR@TPR95: 51.848
	- 1.0
		AUPRC score: 1.748
		FPR@TPR95: 50.763
	- 1.1
		AUPRC score: 1.860
		FPR@TPR95: 50.387
	- 1.2
		AUPRC score: 1.972
		FPR@TPR95: 50.150
	- 1.5
		AUPRC score: 2.286
		FPR@TPR95: 49.456
	- 2.0
		AUPRC score: 2.677
		FPR@TPR95: 48.324
	- 3.0
		AUPRC score: 3.048
		FPR@TPR95: 46.893
	- 5.0
		AUPRC score: 3.252
		FPR@TPR95: 

### Step 3 - Train models with void classifier

In [7]:
models = ["erfnet", "enet", "bisenet"]


In [8]:

savedirs = ["erfnet_training_void", "enet_training_void", "bisenet_training_void"]
pretrained_weights = ["erfnet_pretrained.pth", "enet_pretrained.pth", "bisenetv1_pretrained.pth"]
epochs = 20

# Base directory of the project
base_dir = "../train"
# Dataset directory
data_dir = "../cityscapes"

!cd {base_dir} && python main_v2.py --savedir "bisenet_training_void" --datadir {data_dir} --model "bisenet" --cuda --num-epochs=20 --epochs-save=1 --batch-size=6

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 450MB/s]
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
----- TRAINING - EPOCH 1 -----
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` befo

**Evaluation**

In [ ]:
import torch

no_execute = False
just_once = False

for model in models:
  print("----------------------------")
  for dataset_dir in datasets_list:

    if no_execute:
      break

    load_dir = f'content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{net}_training_void'
    weights = f'/model_best.pth'
    format_file = os.listdir(f'/content/validation_dataset/{dataset_dir}/images')[0].split(".")[1]
    input =f'/content/validation_dataset/{dataset_dir}/images/\*.{format_file}'
    print(f"\nDataset: {dataset_dir} net: {net}")

    if torch.cuda.is_available():
      !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} | tail -n 2
    else:
      !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --cpu | tail -n 2

    if just_once:
      no_execute = True
      just_once = False


**mIoU Void Classification**

In [ ]:
import torch
no_execute = False
just_once = False

for model in models:
  print("----------------------------")

  if no_execute:
      break
  print(f"-----------{model}-------------")
  loadDir = f'content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{model}_training_void'
  weights = f'/model_best.pth'
  if torch.cuda.is_available():
    !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/eval_iou.py --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/ --model {model} | tail -n 25
  else:
    !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/eval_iou.py  --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/  --model {model}  --cpu | tail -n 25


  if just_once:
    no_execute = True
    just_once = False

### Step 4 - Analyze the Effect of Training Loss function

Analyze the effect of the training model along with losses that are specifically made for anomaly detection.

**Losses**

**Fine-tuning**

In [ ]:
# Fine tune ERFNET with different losses
"""
Training:
1. Focal loss
2. LogitNorm+CrossEntropy loss
3. IsoMaxPlus+CrossEntropy loss
4. LogitNorm+Focal loss
5. IsoMaxPlus+Focal loss
"""
titles = ["Focal", "LogitNorm+CrossEntropy", "IsoMaxPlus+CrossEntropy", "LogitNorm+Focal", "IsoMaxPlus+Focal"]
losses = ["Focal", "CrossEntropy", "CrossEntropy", "Focal", "Focal"]
models = ["erfnet", "erfnet", "erfnet_isomaxplus", "erfnet", "erfnet_isomaxplus"]
savedirs = ["erfnet_training_focal_loss", "erfnet_training_logitnorm_cross_entropy_loss", "erfnet_training_isomaxplus_cross_entropy_loss", "erfnet_training_logitnorm_focal_loss", "erfnet_training_isomaxplus_focal_loss"]
logit_normalization_flags = [False, True, False, True, False]
epochs = 20

# Base directory of the project
base_dir = "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/train"
# Dataset directory
data_dir = "/content/cityscapes"
pretrained_weights = "erfnet_pretrained.pth"

# Loop to execute fine-tuning
for title, loss, model, savedir, logit_normalization_flag in zip(titles, losses, models, savedirs, logit_normalization_flags):
    print(f"\n\n----- Fine-tuning with {title} loss -----")
    !cd {base_dir} && python -W ignore main.py --savedir {savedir} --loss {loss} --logit_normalization {logit_normalization_flag} --datadir {data_dir} --model {model} --cuda --num-epochs=20 --epochs-save=1 --FineTune --decoder --loadWeights={pretrained_weights}
    print(f"Model saved in /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{savedir}")
    # zip folder
    !zip -r save_{savedir}.zip /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{savedir}

**Extension Evaluation**

In [ ]:
import torch

no_execute = False
just_once = False

losses = ["CrossEntropy", "Focal", "LogitNorm+CrossEntropy", "IsoMaxPlus+CrossEntropy", "LogitNorm+FocalLoss", "IsoMaxPlus+FocalLoss"]
models = ["erfnet", "erfnet", "erfnet", "erfnet_isomaxplus", "erfnet", "erfnet_isomaxplus"]
load_dirs = ["/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/trained_models/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_focal_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_logitnorm_cross_entropy_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_isomaxplus_cross_entropy_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_logitnorm_focal_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_isomaxplus_focal_loss/"]
weights = ["erfnet_pretrained.pth", "model_best.pth", "model_best.pth", "model_best.pth", "model_best.pth", "model_best.pth"]

for loss, model, load_dir, weight in zip(losses, models, load_dirs, weights):
  print(f"------ Evaluating loss: {loss} ------\n")
  for dataset_dir in ['RoadAnomaly21', 'RoadObsticle21', 'FS_LostFound_full', 'fs_static', 'RoadAnomaly']:
    for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:

      if no_execute:
        break

      format_file = os.listdir(f'/content/Validation_Dataset/{dataset_dir}/images')[0].split(".")[1]
      input =f'/content/Validation_Dataset/{dataset_dir}/images/\*.{format_file}'

      print(f"\nDataset: {dataset_dir} method: {method} loss: {loss}")

      if torch.cuda.is_available():
        !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method  {method} --model {model} --loadDir {load_dir} --loadWeights {weight} | tail -n 2
      else:
        !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method {method}  --model {model} --loadDir {load_dir} --loadWeights {weight} --cpu | tail -n 2

      print("----------------------------")
      if just_once:
        no_execute = True
        just_once = False
    print("----------------------------\n\n")

##Plot

In [ ]:
import torch

# Example image to color
# Nice images: RoadAnomaly/images/28, RoadAnomaly/images/58
input = '/content/Validation_Dataset/RoadAnomaly/images/58.jpg'

### Baseline models ###
for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:
  print(f"Method: {method}")
  save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baseline/{method}'

  if torch.cuda.is_available():
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method  {method} --save-colored {save_image_path}  | tail -n 2
  else:
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method {method} --save-colored {save_image_path} --cpu | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/ccontent/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baseline" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baseline_visualization.png"

### Temperature scaling ###
for t in [0.5, 0.75, 1.1]:
  print(f"Method: MSP, Temperature: {t}")
  save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/temperature/t={t}'

  if torch.cuda.is_available():
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method 'MSP' --temperature {t} --save-colored {save_image_path} | tail -n 2
  else:
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method 'MSP' --cpu --temperature {t} --save-colored {save_image_path} | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/temperature" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/temperature_visualization.png"

### Finetuned models with void ###
for net in ["erfnet", "enet", "bisenet"]:
  save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/void/{net}'
  load_dir = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{net}_training_void'
  weights = f'/model_best.pth'
  print(f"Finetuned network: {net}")
  if torch.cuda.is_available():
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --save-colored {save_image_path} | tail -n 2
  else:
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --cpu --save-colored {save_image_path} | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/void" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/void_visualization.png"

### Losses ###
losses = ["CrossEntropy", "Focal", "LogitNorm", "IsoMaxPlus"]
models = ["erfnet", "erfnet", "erfnet", "erfnet_isomaxplus"]
load_dirs = ["/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/trained_models/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_focal_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_logitnorm_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_isomaxplus_loss/"]
weights = ["erfnet_pretrained.pth", "model_best.pth", "model_best.pth", "model_best.pth"]
for loss, model, load_dir, weight in zip(losses, models, load_dirs, weights):
  for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:
    save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/{loss}/{method}'
    print(f"Method: {method}, loss: {loss}")
    if torch.cuda.is_available():
      !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method  {method} --model {model} --loadDir {load_dir} --loadWeights {weight} --save-colored {save_image_path} | tail -n 2
    else:
      !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method {method}  --model {model} --loadDir {load_dir} --loadWeights {weight} --save-colored {save_image_path} --cpu | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/CrossEntropy" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_CrossEntropy_visualization.png"
!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/Focal" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_Focal_visualization.png"
!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/LogitNorm" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_LogitNorm_visualization.png"
!python /ccontent/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/IsoMaxPlus" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_IsoMaxPlus_visualization.png"

# Zip the images
!zip -r colored_anomalies.zip /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization